In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-sn/

/content/nmt-kn-sn


In [3]:
! chmod a+x /content/nmt-kn-sn/scripts/data_preparation.sh

In [4]:
! /content/nmt-kn-sn/scripts/data_preparation.sh data.kn data.sn

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 17.20 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 23.23 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [5]:
! chmod a+x /content/nmt-kn-sn/scripts/split_dataset.sh

In [6]:
! /content/nmt-kn-sn/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.sn

In [7]:
%cd ..

/content


In [8]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [9]:
!fairseq-preprocess --source-lang kn --target-lang sn \
--trainpref nmt-kn-sn/train/train \
--validpref nmt-kn-sn/dev/dev \
--testpref nmt-kn-sn/test/test \
--destdir nmt-kn-sn/tokenized.kn-sn \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-06 03:59:43 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:59:43 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-sn/tokenized.kn-sn', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [10]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
!fairseq-train /content/nmt-kn-sn/tokenized.kn-sn \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "BiLSTM model - kn to sn - 6 March 2022"

2022-03-06 04:00:03 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-06 04:00:04 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 04:00:06 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'BiLSTM model - kn to sn - 6 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1

In [12]:
!pwd

/content


In [13]:
!fairseq-generate /content/nmt-kn-sn/tokenized.kn-sn \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-10	-0.0068557304330170155	दर ् शनः मन ् द वेगेन चरति
P-10	-0.0156 -0.0010 -0.0011 -0.0127 -0.0000 -0.0067 -0.0162 -0.0072 -0.0012
S-141	ಸುಧನ ್ ವನು ನಿಧಾನವಾಗಿ ನಡೆಯುತ ್ ತಿದ ್ ದಾನೆ
T-141	सुधन ् वः मन ् द वेगेन चरति
H-141	-0.004137339536100626	सुधन ् वः मन ् द वेगेन चरति
D-141	-0.004137339536100626	सुधन ् वः मन ् द वेगेन चरति
P-141	-0.0025 -0.0002 -0.0009 -0.0063 -0.0000 -0.0054 -0.0108 -0.0097 -0.0015
S-120	ಅನುಜನು ಒಳ ್ ಳೆಯದನ ್ ನು ಕಲಿಯುತ ್ ತಾನೆ
T-120	अनुजः सम ् यक ् गुणनाम ् जानाति
H-120	-0.0068128700368106365	अनुजः सम ् यक ् गुणनाम ् जानाति
D-120	-0.0068128700368106365	अनुजः सम ् यक ् गुणनाम ् जानाति
P-120	-0.0010 -0.0047 -0.0004 -0.0034 -0.0002 -0.0369 -0.0001 -0.0068 -0.0079
S-97	ಅಚ ್ ಯುತನು ಚೆನ ್ ನಾಗಿ ಹಾಡುತ ್ ತಾನೆ
T-97	अच ् युतः सम ् यक ् गायति
H-97	-0.0038334932178258896	अच ् युतः सम ् यक ् गायति
D-97	-0.0038334932178258896	अच ् युतः सम ् यक ् गायति
P-97	-0.0040 -0.0001 -0.0049 -0.0025 -0.0016 -0.0024 -0.0003 -0.0180 -0.0006
S-83	ಅಚ ್